# Пространственный анализ

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from dotenv import load_dotenv

# Загрузка данных
load_dotenv()
DATA_PATH = os.getenv('DATA_PATH')
if DATA_PATH is None:
    raise ValueError("DATA_PATH не найден в .env")
df = pd.read_csv(DATA_PATH)

print(f"Загружено строк: {len(df)}")
print(f"Уникальных point_id: {df['point_id'].nunique()}")

# === Базовая статистика ===
display(df.head(10))
display(df.describe(include='all'))

print("\nРаспределение по менеджерам:")
display(df['manager'].value_counts().sort_index())

print("\nРаспределение n_visits:")
display(df['n_visits'].value_counts().sort_index())

# === Анализ дубликатов point_id ===
duplicated_ids = df[df['point_id'].duplicated(keep=False)]
print(f"\nКоличество дублирующихся point_id: {df['point_id'].duplicated().sum()}")
print(f"Количество уникальных ID с дублями: {duplicated_ids['point_id'].nunique()}")

if len(duplicated_ids) > 0:
    print("\nПримеры дубликатов:")
    display(duplicated_ids.sort_values('point_id').head(12))

    # Функция haversine для расчёта расстояний
    def haversine(lat1, lon1, lat2, lon2):
        R = 6371.0  # км
        lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
        lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)
        dlat = lat2_rad - lat1_rad
        dlon = lon2_rad - lon1_rad
        a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        return R * c * 1000  # в метрах

    # Расстояния между всеми парами дублей
    pair_distances = []
    for pid, group in duplicated_ids.groupby('point_id'):
        if len(group) > 1:
            coords = group[['lat', 'lon']].values
            for i in range(len(coords)):
                for j in range(i + 1, len(coords)):
                    dist = haversine(coords[i][0], coords[i][1], coords[j][0], coords[j][1])
                    pair_distances.append(dist)

    if pair_distances:
        print(f"\nРасстояния между дублями (метры):")
        print(f"  Мин: {min(pair_distances):.1f} м")
        print(f"  Макс: {max(pair_distances):.1f} м")
        print(f"  Среднее: {np.mean(pair_distances):.1f} м")
        print(f"  Медиана: {np.median(pair_distances):.1f} м")
        print(f"  95-й перцентиль: {np.percentile(pair_distances, 95):.1f} м")

# === Группировка по менеджерам и пространственный анализ ===
grouped = df.groupby('manager')
stats = {}

for mgr, group in grouped:
    points = group[['lat', 'lon']].values
    n_points = len(group)
    visits_total = group['n_visits'].sum()

    # Центроид (рекомендуемое депо)
    centroid_lat, centroid_lon = points.mean(axis=0)

    # BBox
    lat_min, lat_max = points[:, 0].min(), points[:, 0].max()
    lon_min, lon_max = points[:, 1].min(), points[:, 1].max()

    km_per_deg_lat = 111.0
    km_per_deg_lon = np.cos(np.radians(centroid_lat)) * 111.0

    bbox_span_km_lat = (lat_max - lat_min) * km_per_deg_lat
    bbox_span_km_lon = (lon_max - lon_min) * km_per_deg_lon
    area_approx_km2 = bbox_span_km_lat * bbox_span_km_lon

    # Ближайшие соседи
    rad_points = np.radians(points)
    nn = NearestNeighbors(n_neighbors=2, metric='haversine').fit(rad_points)
    dists_rad, _ = nn.kneighbors(rad_points)
    dists_km = dists_rad[:, 1] * 6371

    avg_nn_km = dists_km.mean()
    p90_nn_km = np.percentile(dists_km, 90)
    median_nn_km = np.median(dists_km)
    frac_isolated = (dists_km > 2 * median_nn_km).mean()

    # Moran's I для n_visits
    y = group['n_visits'].values.astype(float)
    y_dev = y - y.mean()
    dist_mat_km = pairwise_distances(rad_points, metric='haversine') * 6371
    dist_mat_km[dist_mat_km < 1e-6] = 1e-6
    W = 1.0 / dist_mat_km
    np.fill_diagonal(W, 0)
    S0 = W.sum()
    moran_i = (n_points / S0) * (y_dev @ W @ y_dev) / (y_dev @ y_dev) if S0 > 0 else 0

    # Ripley's K
    r_values = [1, 5, 10]
    ripley_k = []
    density = (n_points - 1) / area_approx_km2 if area_approx_km2 > 0 else 0
    for r in r_values:
        neigh = NearestNeighbors(radius=r / 6371, metric='haversine').fit(rad_points)
        indices = neigh.radius_neighbors(rad_points, return_distance=False)
        counts = np.array([len(idx) - 1 for idx in indices])
        observed = counts.mean()
        expected = np.pi * r**2 * density
        k_est = observed / expected if expected > 0 else 0
        ripley_k.append(round(k_est, 3))

    stats[mgr] = {
        'n_points': n_points,
        'visits_total': int(visits_total),
        'centroid_lat': round(centroid_lat, 6),
        'centroid_lon': round(centroid_lon, 6),
        'bbox_span_km_lat': round(bbox_span_km_lat, 1),
        'bbox_span_km_lon': round(bbox_span_km_lon, 1),
        'area_approx_km2': round(area_approx_km2, 1),
        'avg_nn_km': round(avg_nn_km, 3),
        'p90_nn_km': round(p90_nn_km, 3),
        'frac_isolated': round(frac_isolated, 3),
        'moran_i_visits': round(moran_i, 3),
        'ripley_k_1km': ripley_k[0],
        'ripley_k_5km': ripley_k[1],
        'ripley_k_10km': ripley_k[2],
    }

# Вывод сводки
stats_df = pd.DataFrame.from_dict(stats, orient='index')
stats_df = stats_df.sort_values('n_points', ascending=False)

readable_names = {
    'n_points': 'Точек',
    'visits_total': 'Всего визитов',
    'centroid_lat': 'Центроид (шир.)',
    'centroid_lon': 'Центроид (долг.)',
    'bbox_span_km_lat': 'Размах С-Ю (км)',
    'bbox_span_km_lon': 'Размах З-В (км)',
    'area_approx_km2': 'Площадь (км²)',
    'avg_nn_km': 'Ср. расст. до соседа (км)',
    'p90_nn_km': '90-й перцентиль (км)',
    'frac_isolated': 'Доля изолированных',
    'moran_i_visits': 'Моран I',
    'ripley_k_1km': 'Ripley K (1км)',
    'ripley_k_5km': 'Ripley K (5км)',
    'ripley_k_10km': 'Ripley K (10км)',
}

print("\nСводка по менеджерам:")
display(stats_df[list(readable_names.keys())].rename(columns=readable_names).round(3))

# === Рекомендации ===
print("\n=== РЕКОМЕНДАЦИИ ПО ДЕПО (центроиды менеджеров) ===")
for mgr in stats:
    print(f"Менеджер {mgr}: ({stats[mgr]['centroid_lat']}, {stats[mgr]['centroid_lon']})")

print("\n=== ВЕРХНЯЯ ГРАНИЦА ОХВАТА ===")
working_days_dec = 22  # декабрь 2025
working_days_jan = 16  # январь 2025 (пример)
max_visits_per_day = 12  # из обсуждений
service_time_min = 30
driving_hours = 4

upper_dec = working_days_dec * max_visits_per_day
upper_jan = working_days_jan * max_visits_per_day

print(f"При {max_visits_per_day} визитах/день и {service_time_min} мин/визит:")
print(f"  Декабрь ({working_days_dec} дней): макс {upper_dec} визитов на менеджера")
print(f"  Январь ({working_days_jan} дней): макс {upper_jan} визитов на менеджера")
print(f"Текущие визиты: менеджер 0 — 405, 1 — 326, 2 — 394")
print("Вывод: необходимо отсеивать ~20–35% визитов для реалистичной нагрузки")

print("\n=== ИЗОЛИРОВАННЫЕ ТОЧКИ ===")
print("Доля точек с расстоянием до ближайшего соседа > 2×медианы: 16–22%")
print("Рекомендация: прикреплять к ближайшим кластерам или отсеивать при низком приоритете")

Загружено строк: 633
Уникальных point_id: 387


,point_id,manager,lat,lon,n_visits
0,ID231,0,55.371884,43.846878,1
1,ID235,0,55.358381,43.845061,1
2,ID362,0,55.374786,43.832983,1
3,ID188,0,55.423430,43.813190,1
4,ID338,0,55.381287,43.811452,1
5,ID49,1,55.317955,42.154033,1
6,ID32,1,55.548200,42.064568,1
7,ID290,1,55.317959,42.084073,1
8,ID203,1,55.557407,42.195016,1
9,ID31,1,55.577987,42.033764,1


,point_id,manager,lat,lon,n_visits
count,633,633.000000,633.000000,633.000000,633.000000
unique,387,NaN,NaN,NaN,NaN
top,ID194,NaN,NaN,NaN,NaN
freq,2,NaN,NaN,NaN,NaN
mean,NaN,0.995261,56.197952,43.742971,1.777251
std,NaN,0.821955,0.273889,0.450543,0.416420
min,NaN,0.000000,54.906415,41.996456,1.000000
25%,NaN,0.000000,56.231732,43.790293,2.000000
50%,NaN,1.000000,56.262912,43.883837,2.000000
75%,NaN,2.000000,56.311606,43.963304,2.000000



Распределение по менеджерам:


manager
0    215
1    206
2    212
Name: count, dtype: int64

...


Количество дублирующихся point_id: 246
Количество уникальных ID с дублями: 246

Примеры дубликатов:


,point_id,manager,lat,lon,n_visits
58,ID0,0,56.312838,44.020590,2
166,ID0,0,56.312066,44.015611,2
317,ID10,1,56.256701,43.850027,2
415,ID10,1,56.254724,43.852526,2
439,ID100,2,56.314990,43.897989,2
539,ID100,2,56.316066,43.895505,2
542,ID101,2,56.321285,43.917338,2
442,ID101,2,56.316690,43.920028,2
141,ID102,0,56.204917,42.673932,2
163,ID102,0,56.202938,42.674753,2



Расстояния между дублями (метры):
  Мин: 36.7 м
  Макс: 863.9 м
  Среднее: 354.2 м
  Медиана: 328.6 м
  95-й перцентиль: 731.0 м

Сводка по менеджерам:


,Точек,Всего визитов,Центроид (шир.),Центроид (долг.),Размах С-Ю (км),Размах З-В (км),Площадь (км²),Ср. расст. до соседа (км),90-й перцентиль (км),Доля изолированных,Моран I,Ripley K (1км),Ripley K (5км),Ripley K (10км)
0,215,405,56.230,43.863,109.5,95.5,10448.1,0.309,0.580,0.163,0.030,101.196,25.657,12.524
2,212,394,56.227,43.744,192.6,66.9,12891.4,0.326,0.635,0.217,0.099,97.606,34.654,16.011
1,206,326,56.135,43.616,114.6,125.3,14363.8,0.372,0.781,0.204,0.083,161.969,77.771,31.179



=== РЕКОМЕНДАЦИИ ПО ДЕПО (центроиды менеджеров) ===
Менеджер 0: (56.229994, 43.863119)
Менеджер 1: (56.134781, 43.61604)
Менеджер 2: (56.22684, 43.744462)

=== ВЕРХНЯЯ ГРАНИЦА ОХВАТА ===
При 12 визитах/день и 30 мин/визит:
  Декабрь (22 дней): макс 264 визитов на менеджера
  Январь (16 дней): макс 192 визитов на менеджера
Текущие визиты: менеджер 0 — 405, 1 — 326, 2 — 394
Вывод: необходимо отсеивать ~20–35% визитов для реалистичной нагрузки

=== ИЗОЛИРОВАННЫЕ ТОЧКИ ===
Доля точек с расстоянием до ближайшего соседа > 2×медианы: 16–22%
Рекомендация: прикреплять к ближайшим кластерам или отсеивать при низком приоритете
